Meant to work with sequential vector indices and some prompting as per instructions
here prompts/user_per_speaker_01.md to create action items per speaker. 

Here we are summarizing people, and what they said. May not be great for interwoven conversations.

In [ ]:
import json
import os
from datetime import datetime

from langchain_core.prompts import PromptTemplate
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pathlib import Path

import whisperx_transcribe.utils as ut
from whisperx_transcribe import logger, start_time

In [ ]:
HISTORY = "../.history"
PROMPT_FILE = "../prompts/instructions_01.md"
SEG_FILE = "../assets/sample/audio_diarized.json"
GROUP_SEG_FILE = f"../{SEG_FILE.split('.')[0]}_grouped.json"

No grouping is necessary, the transcript is fed into the index step by step.

In [ ]:
segments = ut.load_segments(segments_file=SEG_FILE)
with open(PROMPT_FILE, "r") as f:
    instructions = f.read()

Still a good idea to group the continuous sections of the transcript to make it easy for retriever. 

In [ ]:
grouped_segments = ut.group_speakers(segments=segments)

In [ ]:
len(segments), len(grouped_segments)

In [ ]:
ut.speaker_segment(grouped_segments[0])

Each Document in LlamaIndex will here represent a single turn or a small group of related turns.

In [ ]:
SPEAKERS = ut.get_all_speakers(segments=grouped_segments)
SPEAKERS

In [ ]:
model = "gemma3:4b"
llm = Ollama(model=model, request_timeout=1000)
embed_model = HuggingFaceEmbedding(
    "Qwen/Qwen3-Embedding-0.6B",
    trust_remote_code=True,
    cache_folder="../model",
)
Settings.embed_model = embed_model
Settings.llm = llm

persist_dir = "storage"
Path(persist_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, SummaryIndex

summary_index = SummaryIndex.from_documents(documents=[], show_progress=True)
# vector_index = VectorStoreIndex.from_documents(documents=[], show_progress=True)
# storage_context=StorageContext.from_defaults(persist_dir=persist_dir)
summary_index.set_index_id(f"summary_index_{start_time}")
summary_index.storage_context.persist(persist_dir)
len_segments = len(grouped_segments)
for idx, segment in enumerate(grouped_segments):
    doc = ut.speaker_segment(segment, turn_id=idx)
    summary_index.insert(doc)

In [ ]:
len(summary_index.docstore.docs)

In [ ]:
summary_index.docstore.docs[list(summary_index.docstore.docs.keys())[2]]

In [ ]:
from llama_index.core.vector_stores import (
    FilterOperator,
    MetadataFilter,
    MetadataFilters,
    ExactMatchFilter,
)
from llama_index.core.prompts import RichPromptTemplate, SelectorPromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine

In [ ]:
from llama_index.core.tools import QueryEngineTool
from whisperx_transcribe import query_engines

Multiple qeuery tools that focus on different aspects of the conversation. 

In [ ]:
QUERY_ENGINE_TOOLS = []

Basic per speaker extraction.

In [ ]:
SPEAKERS

In [ ]:
for speaker in SPEAKERS:
    _, qe_speaker_summary_tool = query_engines.query_per_speaker(
        summary_index=summary_index, speaker=speaker
    )
    QUERY_ENGINE_TOOLS.append(qe_speaker_summary_tool)

In [ ]:
QUERY_ENGINE_TOOLS

For the main action items from the conversation (including all participants.)

In [ ]:
_, action_items_tool = query_engines.query_action_items(summary_index)
_, disagreements_tool = query_engines.query_disagreements(summary_index)
_, decisions_tool = query_engines.query_key_decisions(summary_index)
_, key_topics_tool = query_engines.query_key_items(summary_index)
_, tldr_tool = query_engines.query_tldr(summary_index)

QUERY_ENGINE_TOOLS.extend(
    [
        action_items_tool,
        disagreements_tool,
        decisions_tool,
        key_topics_tool,
        tldr_tool,
    ]
)

In [ ]:
QUERY_ENGINE_TOOLS

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMMultiSelector, LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=QUERY_ENGINE_TOOLS,
)

In [ ]:
response = query_engine.query(
    "You are an expert meeting assistant. "
    "Based on the following key conversational exchanges from a meeting transcript, "
    "generate a concise narrative summary. "
    "Describe the flow of the conversation, "
    "how ideas were built upon, and what conclusions were reached."
)
response

In [ ]:
print(response.response)

Even explains why a particular tool was used.

In [ ]:
response.metadata["selector_result"].reasons

Or can do it systematically in a template filling form.

In [ ]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=QUERY_ENGINE_TOOLS,
)

In [ ]:
action_items = query_engine.query("List all action items from the meeting.")
decisions = query_engine.query("List all key decisions made in the meeting.")
topics = query_engine.query("What were the key topics of the meeting?")
disagreements = query_engine.query("Were there any disagreements? List them.")
tldr = query_engine.query("Give a one-paragraph TL;DR of the meeting.")

Final synthesis:

In [ ]:
final_synthesis = f""" 
{tldr}
 
{topics}
 
{decisions}
 
{disagreements}
 
{action_items} 
"""

In [ ]:
from IPython.display import Markdown

In [ ]:
Markdown(final_synthesis)

In [ ]:
# response = llm.complete(final_prompt)

In [ ]:
# print(response.text)